A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that the dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

I want `customer_train.csv` to be saved in a much more storage-efficient way.

## customer_train.csv

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's work relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total work experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

In [1]:
import pandas as pd
ds_jobs = pd.read_csv('./datasets/customer_train.csv')
ds_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  int64  
dtypes: float64(1), int64(3), object(10)
me

In [2]:
# I explore non-numerical columns unique values
dicc = {col:[ds_jobs[col].unique()] for col in ds_jobs.columns if ds_jobs[col].dtype == 'object'}
pd.DataFrame(dicc)

,city,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job
0,"[city_103, city_40, city_21, city_115, city_16...","[Male, nan, Female, Other]","[Has relevant experience, No relevant experience]","[no_enrollment, Full time course, nan, Part ti...","[Graduate, Masters, High School, nan, Phd, Pri...","[STEM, Business Degree, nan, Arts, Humanities,...","[>20, 15, 5, <1, 11, 13, 7, 17, 2, 16, 1, 4, 1...","[nan, 50-99, <10, 10000+, 5000-9999, 1000-4999...","[nan, Pvt Ltd, Funded Startup, Early Stage Sta...","[1, >4, never, 4, 3, 2, nan]"


In [3]:
# So I can classify in nominal and ordered categorical columns
cat_nom = ['city','gender','major_discipline','company_type']
cat_order = ['relevant_experience','enrolled_university','education_level','experience','company_size','last_new_job']

# I establish a natural order for categorical ordered columns
relevant_experience_order = ["No relevant experience","Has relevant experience"]
enrolled_university_order = ["no_enrollment","Part time course","Full time course"]
education_level_order = ["Primary School","High School","Graduate","Masters","Phd"]
experience_order = ["<1"] + [str(i) for i in range(1,21)] + [">20"]
# List of lists
cs_bins = [[f'{10**i}-{((10**(i+1))//2)-1}',f'{(10**(i+1))//2}-{((10**(i+1))-1)}'] for i in range(1,4)]
# Then we merge in an unique list
company_size_order = ["<10"] + [rango for two_rangos in cs_bins for rango in two_rangos] + ["10000+"]
last_new_job_order = ["never"] + [str(i) for i in range(1,5)] + [">4"]

In [4]:
# In this cell I change dtype for each column
for col in ds_jobs.columns:
    if ds_jobs[col].dtype == 'int':
        ds_jobs[col] = ds_jobs[col].astype('int32')
    elif ds_jobs[col].dtype == 'float':
        ds_jobs[col] = ds_jobs[col].astype('float16')
    
    if col in cat_nom:
        ds_jobs[col] = ds_jobs[col].astype('category')
    elif col in cat_order:
        cat_order_dtype = pd.CategoricalDtype(globals()[col+'_order'],ordered=True)
        ds_jobs[col] = ds_jobs[col].astype(cat_order_dtype)
        
ds_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              19158 non-null  int64   
 1   city                    19158 non-null  category
 2   city_development_index  19158 non-null  float16 
 3   gender                  14650 non-null  category
 4   relevant_experience     19158 non-null  category
 5   enrolled_university     18772 non-null  category
 6   education_level         18698 non-null  category
 7   major_discipline        16345 non-null  category
 8   experience              19093 non-null  category
 9   company_size            13220 non-null  category
 10  company_type            13018 non-null  category
 11  last_new_job            18735 non-null  category
 12  training_hours          19158 non-null  int64   
 13  job_change              19158 non-null  int64   
dtypes: category(10), float

In [5]:
# Storing columns with categorical ordered data as a categorical ordered dtype
# allows me to filter in a much more efficient way. Here's an example:
ds_jobs_clean = ds_jobs[(ds_jobs['experience'] >= '10')&(ds_jobs['company_size'] >= "1000-4999")]
ds_jobs_clean

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
9,699,city_103,0.919922,NaN,Has relevant experience,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,0
12,25619,city_61,0.913086,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,3,23,0
31,22293,city_103,0.919922,Male,Has relevant experience,Part time course,Graduate,STEM,19,5000-9999,Pvt Ltd,>4,141,0
34,26494,city_16,0.910156,Male,Has relevant experience,no_enrollment,Graduate,Business Degree,12,5000-9999,Pvt Ltd,3,145,0
40,2547,city_114,0.925781,Female,Has relevant experience,Full time course,Masters,STEM,16,1000-4999,Public Sector,2,14,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19097,25447,city_103,0.919922,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,>4,57,0
19101,6803,city_16,0.910156,Male,Has relevant experience,no_enrollment,High School,NaN,10,10000+,Pvt Ltd,1,89,0
19103,32932,city_10,0.895020,Male,Has relevant experience,Part time course,Masters,Other,>20,1000-4999,Pvt Ltd,>4,18,0
19128,3365,city_16,0.910156,NaN,Has relevant experience,no_enrollment,Graduate,Humanities,>20,1000-4999,Pvt Ltd,>4,23,0
